# Student

In [68]:
import numpy as np

# =============================
class Student:
    def __init__(self, subgenerajs):
        self.skills = {s:0 for s in subjs}
            
    def learn(self, skill):
        lr = .1
        v = self.skills.get(skill, 0)
        self.skills[skill] = min(v+lr, 1) 

    def forget(self):
        fr = .1
        for k,v in self.skills.items():
            self.skills[k] = v*(1-fr)


# =============================
def make_add_subjects(num):
    return ["%d+%d=%d" % (i+1,j+1,i+j+2) for i in range(num) for j in range(num)]

def split_subjects(subjs, num):
    g = [[] for _ in range(num)]
    for s in subjs:
        i = np.random.randint(num)
        g[i].append(s)
    return g
            
# =============================
subjs = make_add_subjects(3)
similar = split_subjects(subjs, 3)
print(similar)


[['3+3=6'], ['1+2=3', '1+3=4', '3+1=4'], ['1+1=2', '2+1=3', '2+2=4', '2+3=5', '3+2=5']]
